In [0]:
import pandas as pd
import numpy as np

# Passo 1: Carregar os teus dados de estatísticas agregadas
# Numa situação real, irias carregar o teu ficheiro CSV completo aqui.
# Ex: estatisticas_df = pd.read_csv('teu_ficheiro_de_estatisticas.csv')

estatisticas_raw_data = {
    'ano': [2024, 2024], 'mes': [1, 1], 'pag_idade': ['até 19 anos', 'até 19 anos'],
    'rec_idade': ['até 19 anos', 'entre 20 e 29 anos'], 'pag_id_regiao': [3, 3], 'rec_id_regiao': [3, 3],
    'total_valor': [677398447.24, 362783705.28], 'total_quantidade': [7897478, 6027016],
    'ticket_medio': [85.77, 60.19], 'ticket_minimo': [33.141701377, 15.9],
    'ticket_maximo': [224.936940104, 69.935618642], 'ticket_medio_aritmetico': [92.773484262, 42.96525269],
    'ticket_mediana': [78.452675855, 20.268407423], 'ticket_p25': [51.069360538, 17.961196698],
    'ticket_p75': [88.637153386, 67.114241762], 'ticket_iiq': [37.567792847, 49.153045064],
    'ticket_desvio_padrao': [62.098824941, 24.975397616]
}
estatisticas_df = pd.DataFrame(estatisticas_raw_data)

# Passo 2: Definir um fator de escala para controlar o tamanho da amostra
# 0.001 significa que vamos gerar 0.1% do total de transações.
fator_escala = 0.001 

# Lista para armazenar os DataFrames de transações geradas para cada grupo
lista_de_dfs_sinteticos = []

print("Iniciando a geração de dados sintéticos com NumPy...")

# Passo 3: Iterar por cada linha (grupo) da tabela de estatísticas
for index, stats_row in estatisticas_df.iterrows():
    
    # Calcular quantas transações gerar para este grupo
    total_original = stats_row['total_quantidade']
    num_a_gerar = int(total_original * fator_escala)

    if num_a_gerar == 0:
        continue

    print(f"Gerando {num_a_gerar} transações para o grupo {index}...")

    # Criar um DataFrame base com as colunas de categoria repetidas
    grupo_sintetico = pd.DataFrame({
        'ano': [stats_row['ano']] * num_a_gerar,
        'mes': [stats_row['mes']] * num_a_gerar,
        'pag_idade': [stats_row['pag_idade']] * num_a_gerar,
        'rec_idade': [stats_row['rec_idade']] * num_a_gerar,
        'pag_id_regiao': [stats_row['pag_id_regiao']] * num_a_gerar,
        'rec_id_regiao': [stats_row['rec_id_regiao']] * num_a_gerar,
    })

    # O coração do método: Gerar os valores das transações usando interpolação de quantis
    quantis = [0, 0.25, 0.5, 0.75, 1.0]
    valores_nos_quantis = [
        stats_row['ticket_minimo'],
        stats_row['ticket_p25'],
        stats_row['ticket_mediana'],
        stats_row['ticket_p75'],
        stats_row['ticket_maximo']
    ]
    
    random_quantiles = np.random.rand(num_a_gerar)
    valores_transacoes = np.interp(random_quantiles, quantis, valores_nos_quantis)
    
    grupo_sintetico['valor_transacao'] = valores_transacoes
    
    lista_de_dfs_sinteticos.append(grupo_sintetico)

# Passo 4: Combinar todos os DataFrames gerados num só
transacoes_individuais = pd.concat(lista_de_dfs_sinteticos, ignore_index=True)

print("\n--- Amostra das Transações Individuais Geradas ---")
print(transacoes_individuais.head())
print(f"\nTotal de {len(transacoes_individuais)} transações sintéticas geradas.")

# Passo 5 (Opcional, mas recomendado): Verificação das estatísticas
print("\n--- Verificando se as estatísticas geradas correspondem às originais ---")
colunas_grupo = ['ano', 'mes', 'pag_idade', 'rec_idade', 'pag_id_regiao', 'rec_id_regiao']
verificacao = transacoes_individuais.groupby(colunas_grupo)['valor_transacao'].agg(
    ['count', 'min', lambda x: x.quantile(0.25), 'median', lambda x: x.quantile(0.75), 'max', 'mean', 'std']
).reset_index()
print(verificacao.round(2))


# Passo 6: Salvar o resultado final num ficheiro CSV

print("\nFicheiro 'transacoes_individuais_numpy.csv' guardado com sucesso!")

In [0]:
display(transacoes_individuais)